In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
  
# adding the root to the path
path_to_add = r'/home/etaylor/code_projects/dubby'  
  
# Add the directory to sys.path

sys.path.insert(0, path_to_add)

In [2]:
import torch

print(torch.cuda.is_available())  # Should print True if CUDA is set up correctly

print(torch.version.cuda)  # Should print '11.4' matching the CUDA version supported by the driver

print(torch.cuda.get_device_name(0))  # Should print the name of your GPU

True
11.8
NVIDIA GeForce RTX 2080 Ti


In [3]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('csv', data_files={'train': '/home/etaylor/code_projects/dubby/data/train.csv',
                                          'test': '/home/etaylor/code_projects/dubby/data/test.csv'})


In [4]:
from transformers import AutoTokenizer
model_version = 'FacebookAI/roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_version)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True,padding=True)

tokenized_data = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/3943 [00:00<?, ? examples/s]

In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_version,
    num_labels=2
    )

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall
    }


training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [7]:
# Start training
trainer.train()

/home/etaylor/.conda/envs/dubby/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss


TrainOutput(global_step=370, training_loss=0.6300248017182222, metrics={'train_runtime': 178.9821, 'train_samples_per_second': 66.085, 'train_steps_per_second': 2.067, 'total_flos': 3112077562798080.0, 'train_loss': 0.6300248017182222, 'epoch': 1.0})

In [ ]:
# to free GPU memory run this line
torch.cuda.empty_cache()

### Evaluate the Model

In [9]:
# eval on the test set that was initiate with the trainer
evaluation_results = trainer.evaluate()
print(evaluation_results)

/home/etaylor/.conda/envs/dubby/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.6094698309898376, 'eval_accuracy': 0.6259193507481613, 'eval_f1': 0.626298454522422, 'eval_precision': 0.7379104477611941, 'eval_recall': 0.5440140845070423, 'eval_runtime': 22.3621, 'eval_samples_per_second': 176.325, 'eval_steps_per_second': 5.545, 'epoch': 1.0}
